In [96]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [97]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [98]:
import random
train_data, valid_data = train_data.split(random_state = random.seed(SEED))


In [99]:
print(vars(train_data.examples[0]).get('text'))
for i in range(len(train_data)):
  vars(train_data.examples[i]).get('text').reverse()
print(vars(train_data.examples[0]).get('text'))

['I', 'stopped', 'by', 'BB', 'and', 'picked', 'up', '4', 'zombie', 'flicks', 'to', 'watch', 'over', 'the', 'weekend', '.', 'Now', ',', 'I', 'understand', 'that', 'the', 'effects', 'will', 'be', 'cheesy', ',', 'the', 'acting', 'will', 'be', 'sub', '-', 'par', ',', 'and', 'the', 'sets', 'will', 'be', 'suspect', '.', 'So', 'I', "'m", 'not', 'expecting', 'much', '.', 'But', 'it', 'should', 'at', 'least', 'have', 'a', 'story', '.', 'Stories', 'do', "n't", 'cost', 'a', 'thing', 'except', 'time', '.....', 'apparently', ',', 'they', 'did', "n't", 'have', 'any', 'time', 'either.<br', '/><br', '/>"Zombie', 'Nation', '"', 'had', '5', 'zombies', 'that', 'appeared', 'near', 'the', 'end', 'of', 'the', 'movie', 'that', 'all', 'looked', 'like', 'new', 'wave', 'hookers', '.', 'The', 'picture', 'of', 'the', 'zombie', 'on', 'the', 'front', 'cover', 'NEVER', 'appears', 'in', 'the', 'movie', '.', 'It', 'was', 'absolutely', 'agonizing', 'to', 'watch', 'and', 'had', 'nothing', 'to', 'offer', 'the', 'genre.<b

In [100]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [101]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [102]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.n_layers=n_layers
        self.rnns = nn.ModuleList()
        for i in range(n_layers):
            input_size = embedding_dim if i == 0 else hidden_dim*2
            self.rnns.append(nn.LSTM(input_size, hidden_dim, num_layers=1, bidirectional=bidirectional, dropout=dropout))
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        text_lengths=text_lengths.cpu()
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        outputs = []
        sent_variable=packed_embedded
        for i in range(self.n_layers):
            packed_output, (hidden, cell) = self.rnns[i](sent_variable)
            sent_variable = packed_output
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [103]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [104]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,387,817 trainable parameters


In [105]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [106]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.9597,  0.8905, -0.7076,  ...,  0.3940, -1.2075, -0.9683],
        [-0.3404,  0.2269,  0.0731,  ..., -0.4427,  0.6267,  0.2811],
        [ 0.7507, -1.9179,  2.2029,  ..., -1.5966,  0.8308, -0.1398]])

In [107]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.9597,  0.8905, -0.7076,  ...,  0.3940, -1.2075, -0.9683],
        [-0.3404,  0.2269,  0.0731,  ..., -0.4427,  0.6267,  0.2811],
        [ 0.7507, -1.9179,  2.2029,  ..., -1.5966,  0.8308, -0.1398]])


In [108]:

import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [109]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [110]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [111]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [112]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [113]:

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [116]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 56s
	Train Loss: 0.355 | Train Acc: 85.41%
	 Val. Loss: 0.387 |  Val. Acc: 83.82%
Epoch: 02 | Epoch Time: 0m 56s
	Train Loss: 0.247 | Train Acc: 90.54%
	 Val. Loss: 0.331 |  Val. Acc: 86.63%
Epoch: 03 | Epoch Time: 0m 56s
	Train Loss: 0.183 | Train Acc: 93.04%
	 Val. Loss: 0.341 |  Val. Acc: 86.05%
Epoch: 04 | Epoch Time: 0m 56s
	Train Loss: 0.142 | Train Acc: 95.02%
	 Val. Loss: 0.325 |  Val. Acc: 86.99%
Epoch: 05 | Epoch Time: 0m 56s
	Train Loss: 0.118 | Train Acc: 95.89%
	 Val. Loss: 0.348 |  Val. Acc: 86.86%


In [117]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Test Loss: 0.389 | Test Acc: 84.53%
